In [1]:
import numpy as np
import itertools
from random import randint
from IPython.display import clear_output
from scipy.stats import entropy
from scipy.special import comb
from math import log, factorial, floor, ceil, isinf, isnan

from Searchspace_Representations import *
from Entropy_Functions import *
from Distributions import *
from Evaluate import *
from Randset import *
from Print import *
from Eps import *

from LPFulleps import CenteredBinomialFulleps as binomialeps #syntax binomialeps[eta, d]
from LPHalfeps import CenteredBinomialHalfeps as binomialeps_rep2
#centered binomial distribution is called with centered_binom_dist(eta)

from May_Results import maydist, mayeps, ternaryeps #syntax ternaryeps[omega, d]

from Uniformeps import uniformoptieps as uniformeps #syntax uniformeps[eta, d]
#Uniform distribution is called with unidist(eta)

from Bliss import blissdist, blisseps #syntax blisseps[k, d]

# distributions are denoted with arrays of the form [w_0,w_1,...,w_eta].
# distributions need to be of length at least 5, filled up w/ zeroes if necessary.

# parameter sets are stored as dictionaries (referred to as eps). For example, to call upon
# level 3 parameter epsilon_20^{(3)}, you call eps[20, 3].
# If epsilon is a Rep-2 parameter set, eps[30, j], eps[31, j], eps[32, j], eps[33, j]
# exist but are constantly set to 0.0 .

# Each set of parameters can be called upon by utilizing their respective syntax.
# The respective syntaxes differ, depending on the distribution.

# To evaluate a certain distribution "dist" with parameter set "eps" for depth "d", call
# for runtimes: evl_T(dist, d, eps)
# for memory size: evl_L(dist, d, eps)
# the returned array gives you the calculated values PER LEVEL, i.e. 
# evl_T(dist, d, eps)[0] returns the calculated runtime on level 0

# For example, to verify our results for B(3), call
# evl_T(etadist(3),7,binomialeps[3, 7])

In [2]:
def optimizer(dist, #the only required input asks for the distribution where an optimized parameter set should be found
              d=4, #asks for the depth of the algorithm tree
              rough=10, #rough trades off initialization time of the initeps for thoroughness of the starting point
              gamma=0.001, # in the gd_search round, gamma defines the step size
              delta=0.0001, # delta defines the minimal amount of optimization per gd_search round that has to be reached before the algorithm terminates
              setupdelta=0.0001, # setupdelta says that, given an optimal value optT, what is the minimal improvement that should be made to continue down a given path. Trades off runtime for thoroughness
              gd_delta=0, # same as setupdelta for the gd_search round
              gd_scale=1, # can be used instead of gd_delta for the minimal optimization to scale linearly with the optimal value optT
              gd_round=True, # if set to false, no gd_search round takes place. Should generally be left True
              initeps={}, # if initeps={}, the program takes a setup step for a decent initialization of the output eps. If a decent parameter set is known, it can be used as a starting point for the gd_search round instead, omitting the setup phase
              f=ideps, # f describes the function that modifies the parameter set after each gd_search round. It might be appropriate to set f to roundeps
              lightningiter=1000000, #this defines the upper bound of parameter sets checked when entering the lightning gd_search round. If the lightning gd_search round is not wished for, this should be set to 0 instead
              optruthdict={}, # if left empty, every parameter will be optimized. makes runtimes inconceivable when working with depth >3. In general, it is recommended to only optimize up to four parameters per iteration; this can be achieved by setting optruthdict to either bintruthdict or fullepstruthdict with randomized inputs (where each level has hamming weight at most 4)
              printset=[] # printset gives the observer the opportunity to see which inputs are currently under observation. If left empty, this does nothing. Does not impact the algorithm itself
             ):
    global setupset
    global gd_set
    gd_set={}
    if optruthdict=={}:
        optimizedict=truthdict(d)
    else:
        optimizedict=copyeps(optruthdict)
    for i in [10,20,21,22,30,31,32,33]:
        for j in range(1,d):
            if optimizedict[i,j]:
                gd_set[i,j]=[-gamma,0,gamma]
            else:
                gd_set[i,j]=[0]
    # throughout this program, initeps always refers to the epsilon eps that currently is spectated.
    # initeps can be already predefined, otherwise the first step is to find a starting point. This is a hybrid approach with brute force (setup) and greedy discrete gradient descent (gd_search).
    if initeps=={}:
        opteps=nulleps(d-1)
        #initialize Dictionary for epsilon's eps and find the first optimal T-value. Should a library at any point prove to be worse than the current optimal value, it can be discarded immediately. During the course of this program, optT never increases
        initeps=nulleps(d-1)
        optT=max(evl_T(dist, d,opteps))
        if rough>0:
            setupset={}
            for i in [10,20,21,22,30,31,32,33]:
                for j in range(1,d):
                    if optimizedict[i,j]:
                        setupset[i,j]=range(rough+1)
                    else:
                        setupset[i,j]=[0]
            # the setup round finds a starting point for the gd_search round. Should an initeps be given or no setup Round be asked for (i.e. when rough=0), this step is skipped
            [optT,opteps]=setup([dist]+d*[0],(d+1)*[0],[searchspace(dist)]+d*[0], (d+1)*[0], (d+1)*[0], d, 1, optT, nulleps(d-1), nulleps(d-1), rough=rough, setupdelta=setupdelta, printset=printset)
    else:
        opteps=formateps(initeps,d)
        optT=max(evl_T(dist, d,opteps))
    oldchangeeps=nulleps(d-1)
    while gd_round==True:
        # the actual optimization part. Every round, the current working eps opteps gets improved until the difference of the old and new runtime is too small
        curT=optT
        cureps=copyeps(opteps)
        [optT,opteps,optchangeeps]=gd_search([dist]+d*[5*[0]], (d+1)*[0], [searchspace(dist)]+d*[0], (d+1)*[0], (d+1)*[0], d, 1, optT, opteps, opteps, oldchangeeps, nulleps(d-1), nulleps(d-1), nulleps(d-1),gamma=gamma, gd_delta=gd_delta, gd_scale=gd_scale, lightningiter=lightningiter, printset=printset)
        oldchangeeps=copyeps(optchangeeps)
        opteps=f(opteps)
        optT=max(evl_T(dist,d,opteps))
        if curT-optT<delta:
            gd_round=False
    return opteps

#The Subroutine for finding a good starting point
def setup(w, R, S, T, L, d, curlevel, optT, opteps, initeps, rough=10, setupdelta=0.0001, printset=[]):
    # setup goes through all possible epsilons (in steps of size 1/rough) and finds the optimal starting point for the gd_search round.
    global setupset
    j=curlevel #leftover from old version
    prevlevel=j-1
    nextlevel=j+1
    for a10 in setupset[10,j]:
        b10=a10/rough
        if w[prevlevel][0]/2<b10:
            break
        for a20 in setupset[20,j]:
            b20=a20/rough
            if (w[prevlevel][0]-2*b10)/2<b20:
                break
            for a21 in setupset[21,j]:
                b21=a21/rough
                if w[prevlevel][1]/2<b21:
                    break
                for a22 in setupset[22,j]:
                    b22=a22/rough
                    if w[prevlevel][2]/2<b22:
                        break
                    for a30 in setupset[30,j]:
                        b30=a30/rough
                        if (w[prevlevel][0]-2*b10-2*b20)/2<b30:
                            break
                        for a31 in setupset[31,j]:
                            b31=a31/rough
                            if (w[prevlevel][2]-2*b21)/2<2*b31:
                                break
                            for a32 in setupset[32,j]:
                                b32=a32/rough
                                if (w[prevlevel][2]-2*b22)/2<b32:
                                    break
                                for a33 in setupset[33,j]:
                                    b33=a33/rough
                                    if w[prevlevel][3]<2*b33:
                                        break
                                    initeps[10,j]=b10
                                    initeps[20,j]=b20
                                    initeps[21,j]=b21
                                    initeps[22,j]=b22
                                    initeps[30,j]=b30
                                    initeps[31,j]=b31
                                    initeps[32,j]=b32
                                    initeps[33,j]=b33
                                    leps=leveleps(initeps,j)
                                    w[j]=cw(w[prevlevel],leps)
                                    S[j]=searchspace(w[j])
                                    R[j]=representations(w[prevlevel],leps)
                                    L[j]=S[j]-R[j]
                                    if j>1:
                                        T[prevlevel]=2*L[j]-(R[j-1]-R[j])
                                    else:
                                        T[prevlevel]=max(0,2*(S[1]-R[1])-1)
                                    for k in range(j,d+1):
                                        T[k]=0
                                    newT=max(T)
                                    if j==d-1:
                                        R[nextlevel]=0
                                        T[nextlevel]=S[j]/2
                                        L[nextlevel]=S[j]/2
                                        T[j]=2*L[nextlevel]-(R[j]-R[nextlevel])
                                        newT=max(T)
                                        if newT<optT:
                                            clear_output(wait=True)
                                            for entry in printset:
                                                print(entry)
                                            print(initeps)
                                            print("T=" + str(T))
                                            print("best runtime is " + str(newT))
                                            optT=newT
                                            opteps=copyeps(initeps)
                                    elif newT<optT-setupdelta:
                                        [optT,opteps]=setup(w, R, S, T, L, d, nextlevel, optT,opteps,initeps, rough=rough, printset=printset)
    return [optT,opteps]

def gd_search(w, R, S, T, L, d,curlevel,optT, opteps, oldeps, oldchangeeps, initeps, changeeps, optchangeeps, gamma=0.001, gd_delta=0, gd_scale=1, lightningiter=100000, printset=[]):
    # one gd_search round defines a greedy and discrete gradient descent optimization step.
    # given the optimal parameter set and the subset of parameters we want to optimize, find the best neighbouring
    # parameter set where all parameters to not be optimized are equal to their current parameter counterpart
    # and all other parameters differ by at most gamma.
    global gd_set
    j=curlevel #leftover from old version
    prevlevel=j-1
    nextlevel=j+1
    for change10 in gd_set[10,j]: 
        if 0>oldeps[10,j]+change10 or w[prevlevel][0]/2<oldeps[10,j]+change10 or change10*oldchangeeps[10,j]<0:
            # check if new parameter is >=0 AND
            # new parameter does not create more values 1 that exist in previous level AND
            # For faster optimization: check if new parameter does not revert changes from previous parameter actualization
            continue
        for change20 in gd_set[20,j]:
            if 0>oldeps[20,j]+change20 or (w[prevlevel][0]-2*(oldeps[10,j]+change10))/2<oldeps[20,j]+change20 or change20*oldchangeeps[20,j]<0:
                # same as with change10
                continue
            for change21 in gd_set[21,j]:
                if 0>oldeps[21,j]+change21 or w[prevlevel][1]/2<oldeps[21,j]+change21 or change21*oldchangeeps[21,j]<0:
                    # same as with change10
                    continue
                for change22 in gd_set[22,j]:
                    if 0>oldeps[22,j]+change22 or w[prevlevel][2]/2<oldeps[22,j]+change22 or change22*oldchangeeps[22,j]<0:
                        # same as with change10
                        continue
                    for change32 in gd_set[32,j]:
                        if 0>oldeps[32,j]+change32 or (w[prevlevel][2]/2-2*(oldeps[22,j]+change22))/2<oldeps[32,j]+change32 or change32*oldchangeeps[32,j]<0:
                            # same as with change10
                            continue
                        for change33 in gd_set[33,j]:
                            if 0>oldeps[33,j]+change33 or w[prevlevel][3]/2<oldeps[33,j]+change33 or change33*oldchangeeps[33,j]<0:
                                # same as with change10
                                continue
                            for change31 in gd_set[31,j]:
                                if 0>oldeps[31,j]+change31 or (w[prevlevel][1]-2*(oldeps[21,j]+change21))/2<oldeps[31,j]+change31 or change31*oldchangeeps[31,j]<0:
                                    # same as with change10
                                    continue
                                for change30 in gd_set[30,j]:
                                    if 0>oldeps[30,j]+change30 or (w[prevlevel][0]-2*(oldeps[20,j]+change20+oldeps[10,j]+change10))/2<oldeps[30,j]+change30 or change30*oldchangeeps[30,j]<0:
                                        # same as with change10
                                        continue
                                    initeps[10,j]=oldeps[10,j]+change10
                                    initeps[20,j]=oldeps[20,j]+change20
                                    initeps[21,j]=oldeps[21,j]+change21
                                    initeps[22,j]=oldeps[22,j]+change22
                                    initeps[30,j]=oldeps[30,j]+change30
                                    initeps[31,j]=oldeps[31,j]+change31
                                    initeps[32,j]=oldeps[32,j]+change32
                                    initeps[33,j]=oldeps[33,j]+change33
                                    changeeps[10,j]=change10
                                    changeeps[20,j]=change20
                                    changeeps[21,j]=change21
                                    changeeps[22,j]=change22
                                    changeeps[30,j]=change30
                                    changeeps[31,j]=change31
                                    changeeps[32,j]=change32
                                    changeeps[33,j]=change33
                                    leps=leveleps(initeps,j)
                                    w[j]=cw(w[j-1],leps)
                                    S[j]=searchspace(w[j])
                                    R[j]=representations(w[prevlevel],leps)
                                    L[j]=S[j]-R[j]
                                    if j>1:
                                        T[prevlevel]=2*L[j]-(R[j-1]-R[j])
                                    else:
                                        T[prevlevel]=max(0,2*(S[1]-R[1])-1)
                                    T[j]=0
                                    for k in range(j+1,d+1):
                                        T[k]=0
                                        R[k]=0
                                        L[k]=0
                                        S[k]=0
                                        w[k]=5*[0]
                                    newT=max(T)
                                    if j==d-1:
                                        R[nextlevel]=0
                                        T[nextlevel]=S[j]/2
                                        L[nextlevel]=S[j]/2
                                        T[j]=2*L[nextlevel]-(R[j]-R[nextlevel])
                                        newT=max(T)
                                        if newT<optT:
                                            clear_output(wait=True)
                                            for entry in printset:
                                                print(entry)
                                            print(initeps)
                                            print("T=" + str(T))
                                            print("best runtime is " + str(newT))
                                            optT=newT
                                            opteps=copyeps(initeps)
                                            optchangeeps=copyeps(changeeps)
                                            notnull=counteps(changeeps)
                                            if notnull>0:
                                                scale=floor(lightningiter**(1/notnull))
                                                if scale>1:
                                                    lightningsets=lightningeps(changeeps,scale,gamma=gamma)
                                                    [optT,opteps,optchangeeps]=lightning_gd([w[0]]+d*[5*[0]], (d+1)*[0], [searchspace(w[0])]+d*[0], (d+1)*[0], (d+1)*[0], d,1, optT, opteps, oldeps, nulleps(d-1), nulleps(d-1), optchangeeps, lightningsets, gamma=gamma, printset=printset)
                                                else:
                                                    clear_output(wait=True)
                                                    for entry in printset:
                                                        print(entry)
                                                    print(initeps)
                                                    print("T=" + str(T))
                                                    print("best runtime is " + str(newT))
                                                    optT=newT
                                                    opteps=copyeps(initeps)
                                                    optchangeeps=copyeps(changeeps)
                                    elif newT<gd_scale*optT-gd_delta:
                                        [optT,opteps,optchangeeps]=gd_search(w, R, S, T, L, d,nextlevel,optT, opteps, oldeps, oldchangeeps, initeps, changeeps, optchangeeps,gamma=gamma, gd_delta=gd_delta, gd_scale=gd_scale, lightningiter=lightningiter, printset=printset)
    return [optT,opteps,optchangeeps]

def lightning_gd(w, R, S, T, L, d,curlevel,optT, opteps, oldeps, initeps, changeeps, optchangeeps, lightningsets, gamma=0.001, printset=[]):
    j=curlevel
    prevlevel=j-1
    nextlevel=j+1
    for change10 in lightningsets[10,j]:
        if 0>oldeps[10,j]+change10 or w[prevlevel][0]/2<oldeps[10,j]+change10:
            continue
        for change20 in lightningsets[20,j]:
            if 0>oldeps[20,j]+change20 or (w[prevlevel][0]-2*(oldeps[10,j]+change10))/2<oldeps[20,j]+change20:
                continue
            for change21 in lightningsets[21,j]:
                if 0>oldeps[21,j]+change21 or w[prevlevel][1]/2<oldeps[21,j]+change21:
                    continue
                for change22 in lightningsets[22,j]:
                    if 0>oldeps[22,j]+change22 or w[prevlevel][2]/2<oldeps[22,j]+change22:
                        continue
                    for change32 in lightningsets[32,j]:
                        if 0>oldeps[32,j]+change32 or (w[prevlevel][2]/2-2*(oldeps[22,j]+change22))/2<oldeps[32,j]+change32:
                            continue
                        for change33 in lightningsets[33,j]:
                            if 0>oldeps[33,j]+change33 or w[prevlevel][3]/2<oldeps[33,j]+change33:
                                continue
                            for change31 in lightningsets[31,j]:
                                if 0>oldeps[31,j]+change31 or (w[prevlevel][1]-2*(oldeps[21,j]+change21))/2<oldeps[31,j]+change31:
                                    continue
                                for change30 in lightningsets[30,j]:
                                    if 0>oldeps[30,j]+change30 or (w[prevlevel][0]-2*(oldeps[20,j]+change20+oldeps[10,j]+change10))/2<oldeps[30,j]+change30:
                                        continue
                                    initeps[10,j]=oldeps[10,j]+change10
                                    initeps[20,j]=oldeps[20,j]+change20
                                    initeps[21,j]=oldeps[21,j]+change21
                                    initeps[22,j]=oldeps[22,j]+change22
                                    initeps[30,j]=oldeps[30,j]+change30
                                    initeps[31,j]=oldeps[31,j]+change31
                                    initeps[32,j]=oldeps[32,j]+change32
                                    initeps[33,j]=oldeps[33,j]+change33
                                    changeeps[10,j]=change10
                                    changeeps[20,j]=change20
                                    changeeps[21,j]=change21
                                    changeeps[22,j]=change22
                                    changeeps[30,j]=change30
                                    changeeps[31,j]=change31
                                    changeeps[32,j]=change32
                                    changeeps[33,j]=change33
                                    leps=leveleps(initeps,j)
                                    w[j]=cw(w[j-1],leps)
                                    S[j]=searchspace(w[j])
                                    R[j]=representations(w[prevlevel],leps)
                                    L[j]=S[j]-R[j]
                                    if j>1:
                                        T[prevlevel]=2*L[j]-(R[j-1]-R[j])
                                    else:
                                        T[prevlevel]=max(0,2*(S[1]-R[1])-1)
                                    for k in range(j,d+1):
                                        T[k]=0
                                    newT=max(T)
                                    if j==d-1:
                                        R[nextlevel]=0
                                        T[nextlevel]=S[j]/2
                                        L[nextlevel]=S[j]/2
                                        T[j]=2*L[nextlevel]-(R[j]-R[nextlevel])
                                        newT=max(T)
                                        if newT<optT:
                                            clear_output(wait=True)
                                            for entry in printset:
                                                print(entry)
                                            print(initeps)
                                            print("T=" + str(T))
                                            print("best runtime is " + str(newT))
                                            optT=newT
                                            opteps=copyeps(initeps)
                                            optchangeeps=normaleps(changeeps, gamma)
                                    elif newT<optT:
                                        [optT,opteps,optchangeeps]=lightning_gd(w, R, S, T, L, d, nextlevel, optT, opteps, oldeps, initeps, changeeps, optchangeeps, lightningsets, gamma=gamma, printset=printset)
    return [optT,opteps,optchangeeps]

In [ ]:
# example for an optimizer call. Finds depth 4 Rep-3 parameters for distribution [0.5,0.25,0,0,0]

dist=maydist[3]

optimizer(dist,
              d=6,
              rough=10,
              gamma=0.001,
              delta=0.0001,
              setupdelta=0.001,
              gd_delta=0, 
              gd_scale=1, 
              gd_round=True, 
              initeps={},
              f=ideps,
              lightningiter=1000000,
              optruthdict=fullbintruthdict(8*'11111111'),
              printset=[]
             )

In [ ]:
# this is the optimization part for CBD
cbdoptieps={}
# randset=[8*'00000000'] #for Rep-0
# randset=[8*'10000000'] #for Rep-1
# randset=[8*'11110000'] #for Rep-2
randset=[8*'11110000']+create_randset(8) #for Rep-3
for eta in range(2,4):
    dist=centered_binom_dist(eta)
    cbdoptieps[eta,1]={}
    for d in range(2,9):
        cbdoptieps[eta,d]=cbdoptieps[eta,d-1]
        for randbits in randset:
            initeps=cbdoptieps[eta,d]
            cbdoptieps[eta,d]=optimizer(dist,rough=0,gamma=0.001,optruthdict=fullbintruthdict(randbits),gd_delta=0, initeps=initeps, d=d,f=ideps, printset=["eta=" + str(eta), "d=" + str(d), randbits])

In [ ]:
# this is the optimization part for uniform distributions
uoptieps={}
# randset=[8*'00000000'] #for Rep-0
# randset=[8*'10000000'] #for Rep-1
# randset=[8*'11110000'] #for Rep-2
randset=[8*'11110000']+create_randset(8) #for Rep-3
for eta in range(1,4):
    dist=unidist(eta)
    uoptieps[eta,1]={}
    for d in range(2,9):
        uoptieps[eta,d]=uoptieps[eta,d-1]
        for randbits in randset:
            initeps=uoptieps[eta,d]
            uoptieps[eta,d]=optimizer(dist,rough=0,gamma=0.001,optruthdict=fullbintruthdict(randbits),gd_delta=0, initeps=initeps, d=d,f=ideps, printset=["eta=" + str(eta), "d=" + str(d), randbits])

In [ ]:
# this is the optimization part of the code for ternary distributions. 
teroptieps={}
# randset=[8*'00000000'] #for Rep-0
# randset=[8*'10000000'] #for Rep-1
# randset=[8*'11110000'] #for Rep-2
randset=[8*'11110000']+create_randset(8) #for Rep-3
for k in range(6):
    dist=maydist[k]
    teroptieps[maydist[k][1],1]={}
    for d in range(2,3):
        teroptieps[maydist[k][1],d]=teroptieps[maydist[k][1],d-1]
        for randbits in randset:
            initeps=teroptieps[maydist[k][1],d]
            teroptieps[maydist[k][1],d]=optimizer(dist,rough=0,gamma=0.001,optruthdict=fullbintruthdict(randbits),gd_delta=0, initeps=initeps, d=d,f=roundeps, printset=["omega=" + str(2*maydist[k][1]), "d=" + str(d), randbits])

In [ ]:
# this is the optimization part for BLISS
blissoptieps={}
# randset=[8*'00000000'] #for Rep-0
# randset=[8*'10000000'] #for Rep-1
# randset=[8*'11110000'] #for Rep-2
randset=[8*'11110000']+create_randset(8) #for Rep-3
for eta in (0,1,3,4):
    dist=blissdist[eta]
    blisseps[eta,1]={}
    for d in range(2,9):
        blisseps[eta,d]=blisseps[eta,d-1]
        for randbits in randset:
            initeps=blisseps[eta,d]
            blisseps[eta,d]=optimizer(dist,rough=0,gamma=0.001,optruthdict=fullbintruthdict(randbits),gd_delta=0, initeps=initeps, d=d,f=roundeps, printset=["BLISS-" + str(eta), "d=" + str(d), randbits])

In [ ]:
# this tool evaluates the runtimes that result in our optimization efforts for CBD
eta=3 # 2,3
d=7 #2,3,...,8

evl_T(centered_binom_dist(eta),d,binomialeps[eta, d]) #binomialeps_rep2 for Rep-2 parameters

In [ ]:
# this tool evaluates the runtimes that result in our optimization efforts for the uniform distribution
eta=3 # 2,3
d=7 #2,3,...,8

evl_T(unidist(eta),d,uniformeps[eta, d])

In [ ]:
# this tool evaluates the runtimes that result in our optimization efforts for ternary distributions
k=5 # 0,1,...,5
d=6 #2,3,...,8

evl_T(maydist[k],d,ternaryeps[maydist[k][1]*2, d])

In [ ]:
# this tool evaluates the runtimes that result in our optimization efforts for BLISS
k=4 #0,1,...,4
d=6 #2,3,...,8

evl_T(blissdist[k],d,blisseps[k, d])

In [ ]:
# Example for Tableprint. Prints a LaTeX-compilable table for Rep-3 Results for centered_binom_dist(3), d=2,...,8

printtop()
for d in range(2,9):
    tableprint("\\mathcal{B}(3)", binomialeps[3, d], eta=3, d=d, dist=centered_binom_dist(3), treatzero=" ", ifthrees=True, ifcolor=True, iftop=False, ifbottom=False)
printbottom()